In [ ]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------------
# 1) Entradas del modelo
# ---------------------------------------------------------------
# Parámetros de un alerón trasero (ajustables)
S_ref = 1.00  # m^2 área de referencia del alerón
CL_alpha_per_deg = 0.085  # pendiente CL por grado (≈ 0.085/deg ~ 4.87 rad^-1)
alpha_stall_deg = 30.0    # ángulo de pérdida/saturación
CL0 = 0.0                 # CL a 0° (puedes usar offset si tu geometría lo requiere)

CD0 = 0.25                # arrastre base
k_induced = 0.16          # factor inducido (CD = CD0 + k*CL^2)

# Rango de ángulos a simular (consistente con F1 moderno)
ANGLE_MIN = 10.0
ANGLE_MAX = 50.0
ANGLE_STEP = 1.0

# Velocidades para cálculo de fuerzas (km/h)
SPEEDS_KMH = [200.0, 300.0, 330.0]

# Cargar datos de entorno (ρ y q de referencia por circuito)
env = pd.read_csv("datos_entorno_circuitos.csv")

# ---------------------------------------------------------------
# 2) Definición de polares CL(α) y CD(α)
# ---------------------------------------------------------------
def cl_raw(alpha_deg):
    # lineal hasta stall, luego saturación suave con tanh
    a = alpha_deg
    cl_lin = CL0 + CL_alpha_per_deg * a
    # saturación hacia CL_max con transición suave
    cl_max = CL0 + CL_alpha_per_deg * alpha_stall_deg
    # transicion: para a>alpha_stall_deg, suaviza hacia cl_max
    # usa una compresión con tanh centrada en alpha_stall_deg
    sharpness = 0.25  # más grande = transición más brusca
    t = np.tanh(sharpness * (a - alpha_stall_deg))
    cl = np.where(a <= alpha_stall_deg, cl_lin, cl_max - (cl_max - cl_lin) * (1 + t) / 2.0)
    # En F1 el alerón trasero genera downforce: CL efectivo negativo respecto al eje z del coche.
    # Para trabajar con |CL| en eficiencia, retornamos CL SIGNADO negativo:
    return -np.abs(cl)

def cd_from_cl(cl):
    return CD0 + k_induced * (cl**2)

def cd_raw(alpha_deg):
    cl = cl_raw(alpha_deg)
    return cd_from_cl(cl)

# ---------------------------------------------------------------
# 3) Generación de dataset: por circuito y ángulo
# ---------------------------------------------------------------
rows = []
angles = np.arange(ANGLE_MIN, ANGLE_MAX + 1e-9, ANGLE_STEP)

for _, row in env.iterrows():
    track = row["track"]
    rho = float(row["air_density_kg_per_m3"])
    # calcular q con rho por velocidad exacta (no usar q precalculada por redondeo)
    for alpha in angles:
        CL = float(cl_raw(alpha))
        CD = float(cd_raw(alpha))
        E = abs(CL) / CD
        # fuerzas por velocidad
        out = {
            "track": track,
            "angle_deg": alpha,
            "CL": CL,
            "CD": CD,
            "E_CL_over_CD": E
        }
        for vk in SPEEDS_KMH:
            v = vk / 3.6
            q = 0.5 * rho * v * v
            DF = q * S_ref * abs(CL)   # downforce [N]
            DR = q * S_ref * CD        # drag [N]
            out[f"DF_{int(vk)}kph_N"] = DF
            out[f"DR_{int(vk)}kph_N"] = DR
        rows.append(out)

df = pd.DataFrame(rows)
df.to_csv("Datos_de_Simulacion.csv", index=False)
print("Generado 'Datos_de_Simulacion.csv' con CL, CD, E y fuerzas por circuito.")

Generado 'Datos_de_Simulacion.csv' con CL, CD, E y fuerzas por circuito.


In [9]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------------
# Entradas
# ---------------------------------------------------------------
NOMBRE_ARCHIVO = "Datos_de_Simulacion.csv"  # CL, CD, por pista y ángulo
GRADO_POLI_MAX = 4
ANGLE_MIN, ANGLE_MAX = 10.0, 50.0

# Velocidades de referencia
VREF_DF_KMH = 200.0   # para beneficio de DF en curvas
VREF_DR_KMH = 330.0   # para penalización de drag en recta
S_REF = 1.0           # m^2 (debe coincidir con el usado al generar datos)

# Pesos por pista (ajusta a gusto)
WE_BY_TRACK = {
    "Monaco": 1.0,
    "Monza":  1.0,
    "Silverstone": 1.0,
    "Spa": 1.0,
    "Interlagos": 1.0,
}
WDF_BY_TRACK = {
    "Monaco": 1.2e-7,
    "Monza":  0.8e-7,
    "Silverstone": 1.0e-7,
    "Spa": 1.0e-7,
    "Interlagos": 1.0e-7,
}
WDR_BY_TRACK = {
    "Monaco": 1.0e-4,
    "Monza":  1.5e-3,   # alto, para favorecer baja carga
    "Silverstone": 6.0e-4,
    "Spa": 5.0e-4,
    "Interlagos": 3.0e-4,
}
DF_TARGET_BY_TRACK = {
    "Monaco": 8000.0,
    "Monza":  4500.0,
    "Silverstone": 6500.0,
    "Spa": 6000.0,
    "Interlagos": 6000.0,
}

# Densidad del aire por pista
ENV_FILE = "datos_entorno_circuitos.csv"  # si falta, usa DEFAULT_RHO
DEFAULT_RHO = 1.20

# Método de respaldo para la raíz de dJ: "bisection" o "regula_falsi"
BACKUP_METHOD = "bisection"

# ---------------------------------------------------------------
# Utilidades de formato (prints lindos)
# ---------------------------------------------------------------
def fmt_deg(x, dec=2):
    return f"{x:,.{dec}f}°".replace(",", "_").replace(".", ",").replace("_", ".")
def fmt_float(x, dec=3):
    return f"{x:,.{dec}f}".replace(",", "_").replace(".", ",").replace("_", ".")
def fmt_newton(x):  # para J y términos (alinear con signo)
    return f"{x: .6f}"
def fmt_N(x):       # fuerza en N con miles
    return f"{int(round(x)):>6,d}N".replace(",", ".")

def print_table_main(rows, vref_df, vref_dr):
    # Tabla principal: Track | α_opt | E | DF@vref_df | DR@vref_dr | J
    headers = ["Track", "α_opt", "E", f"DF@{int(vref_df)}", f"DR@{int(vref_dr)}", "J"]
    data = []
    for r in rows:
        data.append([
            r["track"],
            fmt_deg(r["alpha_deg"], 2),
            fmt_float(r["E"], 3),
            fmt_N(r["DF"]),
            fmt_N(r["DR"]),
            fmt_newton(r["J"]),
        ])
    _print_table(headers, data)

def print_table_terms(rows):
    # Tabla de términos: -wE·E | +wDF·(ΔDF)^2 | +wDR·DR
    headers = ["Track", "-wE·E", "+wDF·(ΔDF)^2", "+wDR·DR"]
    data = []
    for r in rows:
        data.append([
            r["track"],
            fmt_newton(r["term_E"]),
            fmt_newton(r["term_DF"]),
            fmt_newton(r["term_DR"]),
        ])
    _print_table(headers, data)

def print_table_errors(rows):
    # Tabla de modelado y errores (sin α_guess ni |dJ|, como pediste)
    headers = ["Track", "Puntos", "Grado (CL/CD)", "RMSE_CL", "RMSE_CD"]
    data = []
    for r in rows:
        grado_txt = f"{r['grado_CL']}/{r['grado_CD']}"
        data.append([
            r["track"],
            r["n_points"],
            grado_txt,
            fmt_float(r["rmse_CL"], 6),
            fmt_float(r["rmse_CD"], 6),
        ])
    _print_table(headers, data)

def _print_table(headers, data):
    # Cálculo de anchos
    cols = list(zip(*([headers] + data)))
    widths = [max(len(str(x)) for x in col) for col in cols]
    # Separadores
    hline = "+-" + "-+-".join("-" * w for w in widths) + "-+"
    # Header
    print(hline)
    print("| " + " | ".join(f"{h:<{w}}" for h, w in zip(headers, widths)) + " |")
    print(hline)
    # Filas
    for row in data:
        print("| " + " | ".join(f"{str(v):>{w}}" for v, w in zip(row, widths)) + " |")
    print(hline)

# ---------------------------------------------------------------
# Utilidades numéricas (métodos clásicos)
# ---------------------------------------------------------------
def poly_fit(x, y, grado_max):
    x = np.asarray(x, float); y = np.asarray(y, float)
    m = np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    if len(x) < 2:
        raise ValueError("Pocos puntos para ajustar.")
    grado = min(grado_max, len(x)-1)
    return np.poly1d(np.polyfit(x, y, grado)), grado

def bisection(g, a, b, tol=1e-6, maxiter=200):
    fa, fb = g(a), g(b)
    if not np.isfinite(fa) or not np.isfinite(fb) or fa*fb > 0:
        raise ValueError("Bisección: bracket inválido.")
    for _ in range(maxiter):
        c = 0.5*(a+b); fc = g(c)
        if abs(fc) <= tol or 0.5*(b-a) < tol:
            return c
        if fa*fc < 0:
            b, fb = c, fc
        else:
            a, fa = c, fc
    return 0.5*(a+b)

def regula_falsi(g, a, b, tol=1e-6, maxiter=200):
    fa, fb = g(a), g(b)
    if not np.isfinite(fa) or not np.isfinite(fb) or fa*fb > 0:
        raise ValueError("Regula Falsi: bracket inválido.")
    x_old = a
    for _ in range(maxiter):
        x = (a*fb - b*fa)/(fb-fa)
        fx = g(x)
        if abs(fx) < tol or abs(x - x_old) < tol:
            return x
        if fa*fx < 0:
            b, fb = x, fx
        else:
            a, fa = x, fx
        x_old = x
    return x

def choose_backup():
    return bisection if BACKUP_METHOD == "bisection" else regula_falsi

# ---------------------------------------------------------------
# Lectura de datos
# ---------------------------------------------------------------
def load_env_rho():
    try:
        env = pd.read_csv(ENV_FILE)
        m = pd.Series(env.air_density_kg_per_m3.values, index=env.track).to_dict()
        return m
    except Exception:
        return {}

# ---------------------------------------------------------------
# Principal
# ---------------------------------------------------------------
def main():
    df = pd.read_csv(NOMBRE_ARCHIVO)
    tracks = sorted(df['track'].unique().tolist())
    rho_map = load_env_rho()

    results = []  # métricas y objetivo
    results_err = []  # modelado y errores

    for track in tracks:
        dt = df[df.track == track].copy()
        x = dt['angle_deg'].values
        cl = dt['CL'].values
        cd = dt['CD'].values

        # Ajuste polinomial (y RMSE)
        pCL, grado_CL = poly_fit(x, cl, GRADO_POLI_MAX)
        pCD, grado_CD = poly_fit(x, cd, GRADO_POLI_MAX)
        cl_hat = pCL(x); cd_hat = pCD(x)
        rmse_CL = float(np.sqrt(np.mean((cl - cl_hat) ** 2)))
        rmse_CD = float(np.sqrt(np.mean((cd - cd_hat) ** 2)))

        dCL = pCL.deriv()
        dCD = pCD.deriv()

        # Parámetros por pista
        wE  = WE_BY_TRACK.get(track, 1.0)
        wDF = WDF_BY_TRACK.get(track, 1.0e-7)
        wDR = WDR_BY_TRACK.get(track, 5.0e-4)
        DFt = DF_TARGET_BY_TRACK.get(track, 6000.0)
        rho = float(rho_map.get(track, DEFAULT_RHO))

        v_df = VREF_DF_KMH/3.6
        v_dr = VREF_DR_KMH/3.6
        q_df = 0.5 * rho * v_df * v_df
        q_dr = 0.5 * rho * v_dr * v_dr

        # Objetivo
        def E(alpha):
            CL = pCL(alpha); CD = pCD(alpha)
            return abs(CL)/CD

        def DF_ref(alpha):
            return q_df * S_REF * abs(pCL(alpha))

        def DR_330(alpha):
            return q_dr * S_REF * pCD(alpha)

        def J(alpha):
            return -wE*E(alpha) + wDF*(DF_ref(alpha) - DFt)**2 + wDR*DR_330(alpha)

        # Derivadas
        def dE(alpha):
            CL = pCL(alpha); CD = pCD(alpha)
            CLp = dCL(alpha); CDp = dCD(alpha)
            s = 1.0 if CL >= 0 else -1.0
            num = (s*CLp)*CD - abs(CL)*CDp
            den = (CD**2)
            return num/den

        def dDF(alpha):
            CL = pCL(alpha); CLp = dCL(alpha)
            s = 1.0 if CL >= 0 else -1.0
            return q_df * S_REF * s * CLp

        def dDR(alpha):
            return q_dr * S_REF * dCD(alpha)

        def dJ(alpha):
            return -wE*dE(alpha) + 2.0*wDF*(DF_ref(alpha) - DFt)*dDF(alpha) + wDR*dDR(alpha)

        # Búsqueda de bracket y solución
        lo, hi = max(ANGLE_MIN, x.min()), min(ANGLE_MAX, x.max())
        # malla para bracket
        aa = np.linspace(lo, hi, 400)
        gg = np.array([dJ(a) for a in aa])
        bracket = None
        for i in range(len(aa)-1):
            if np.isfinite(gg[i]) and np.isfinite(gg[i+1]) and gg[i]*gg[i+1] <= 0:
                bracket = (aa[i], aa[i+1]); break

        solver = choose_backup()
        if bracket:
            try:
                a_star = solver(dJ, bracket[0], bracket[1])
            except Exception:
                # fallback: mínimo de J en malla
                grid = np.linspace(lo, hi, 2001)
                vals = np.array([J(a) for a in grid])
                a_star = float(grid[np.argmin(vals)])
        else:
            # si no hay raíz interior, elegir el extremo con menor J
            a_star = float(lo if J(lo) < J(hi) else hi)

        # Métricas en el óptimo
        E_star = float(E(a_star))
        DF_star = float(DF_ref(a_star))
        DR_star = float(DR_330(a_star))
        J_star = float(J(a_star))
        term_E  = float(-wE*E_star)
        term_DF = float(wDF*(DF_star - DFt)**2)
        term_DR = float(wDR*DR_star)

        # Resultados para tablas principales
        results.append({
            "track": track,
            "alpha_deg": float(a_star),
            "E": E_star,
            "DF": DF_star,
            "DR": DR_star,
            "J": J_star,
            "term_E": term_E,
            "term_DF": term_DF,
            "term_DR": term_DR,
        })

        # Resultados de errores (sin α_guess ni |dJ|)
        results_err.append({
            "track": track,
            "n_points": int(len(x)),
            "grado_CL": int(grado_CL),
            "grado_CD": int(grado_CD),
            "rmse_CL": rmse_CL,
            "rmse_CD": rmse_CD,
        })

    # Prints ordenados
    print("\nResultados por pista (métricas principales):")
    print_table_main(results, VREF_DF_KMH, VREF_DR_KMH)
    print("\nDesglose de términos del objetivo J(α):")
    print_table_terms(results)
    print("\nModelado y errores:")
    print_table_errors(results_err)

if __name__ == "__main__":
    main()


Resultados por pista (métricas principales):
+-------------+--------+-------+---------+---------+-----------+
| Track       | α_opt  | E     | DF@200  | DR@330  | J         |
+-------------+--------+-------+---------+---------+-----------+
|  Interlagos | 19,24° | 2,406 |  2.880N |  3.258N | -0.455650 |
|      Monaco | 37,39° | 1,709 |  5.976N |  9.521N | -0.264774 |
|       Monza | 10,00° | 2,353 |  1.562N |  1.808N |  1.049911 |
| Silverstone | 16,05° | 2,476 |  2.549N |  2.803N |  0.766392 |
|         Spa | 16,31° | 2,473 |  2.530N |  2.786N |  0.124130 |
+-------------+--------+-------+---------+---------+-----------+

Desglose de términos del objetivo J(α):
+-------------+-----------+--------------+-----------+
| Track       | -wE·E     | +wDF·(ΔDF)^2 | +wDR·DR   |
+-------------+-----------+--------------+-----------+
|  Interlagos | -2.406486 |     0.973303 |  0.977533 |
|      Monaco | -1.708679 |     0.491795 |  0.952110 |
|       Monza | -2.352558 |     0.690368 |  2.712101 